In [1]:
#e
import threading
import queue
import os
import math
import itertools

import datasets as hfds
import torch
import torchvision.transforms.functional as TF
import torchvision.io as TFIO

import minai.sampler as mins
import minai.datasets as minds

In [2]:
#e
def simple_collate_func(array_of_results):
    xs = [r[0] for results in array_of_results for r in results]
    ys = [r[1] for results in array_of_results for r in results]
    return xs, ys

class HFCollate:
    def __init__(self, ds: hfds.Dataset):
        self.features = tuple(ds.features.keys())

    def __call__(self, array_of_results):
        collated = [[] for _ in range(len(self.features))]
        for result in array_of_results:
            for i, feature in enumerate(self.features):
                collated[i].extend(result[feature])
        return collated
    
    def __repr__(self):
        return f"HFCollate(features={self.features})"

In [3]:
#e
WORK_TYPE_NONE = 0
WORK_TYPE_LOAD_BATCHES = 1
WORK_TYPE_GET_ITEMS = 2
WORK_TYPE_SHUTDOWN = 100

class WorkItem:
    def __init__(self, type):
        self.type = type

    def __repr__(self): return f"{self.__class__.__name__}({str(vars(self))})"

class WorkItemLoadBatches(WorkItem):
    def __init__(self, cached_iter: "Collator.CachedIter", num_batches):
        super().__init__(WORK_TYPE_LOAD_BATCHES)
        self.cached_iter = cached_iter
        self.num_batches = num_batches

class WorkItemGetItems(WorkItem):
    def __init__(self, group: "WorkGroup", indices):
        super().__init__(WORK_TYPE_GET_ITEMS)
        self.group = group
        self.indices = indices
        self.results = None

        self.group.gi_work_array.append(self)

class COPTS: # CollatorOpts
    def __init__(self,
                 sampler_iter: mins.SamplerIter = None,
                 getitem_func = None,
                 collate_func = None,
                 *, 
                 num_threads=None, 
                 cached_batch_count=1,
                 sub_batch_divisor=1.0, 
                 debug_out_queue: queue.SimpleQueue = None):
        self.sampler_iter = sampler_iter
        self.getitem_func = getitem_func
        self.collate_func = collate_func
        self.num_threads = num_threads if num_threads is not None else max(1, os.cpu_count()-1)
        self.cached_batch_count = cached_batch_count
        self.sub_batch_divisor = sub_batch_divisor
        self.debug_out_queue = debug_out_queue
        self.sub_batch_size = 0

    def finalize(self):
        if self.num_threads:
            # No divisor seems to show the best result from the very limited testing so far even with multiple threads ¯\_(ツ)_/¯
            self.sub_batch_divisor = max(1.0, self.sub_batch_divisor)
        else:
            self.cached_batch_count = 0
            self.sub_batch_divisor = 1.0
        self.sub_batch_size = math.ceil(self.sampler_iter.opts.batch_size / self.sub_batch_divisor)

        return self

class Collator:
    class CachedIter:
        def __init__(self, serial, iter, num_groups_total):
            self.serial = serial
            self.iter = iter
            self.num_groups_total = num_groups_total
            self.num_groups_requested = 0
            self.num_groups_started = 0
            self.num_groups_finished = 0
            self.lock = threading.Lock()
            self.collated_queue = queue.SimpleQueue() # Not the greatest to have a queue per CachedIter, but it simplifies the DataLoader

    def __init__(self, copts: COPTS):
        self.opts = copts.finalize()

        self.threads_spawned = False

        self.work_queue = queue.SimpleQueue()
        
        self.global_lock = threading.Lock()
        self.current_iter_serial = 0
        self.cached_iters: dict[int, Collator.CachedIter] = {}

    def start_new_iter(self):
        if not self.threads_spawned: 
            for i in range(self.opts.num_threads): threading.Thread(target=collator_threadproc, args=(i+1, self)).start()
            self.threads_spawned = True

        self.global_lock.acquire()
        
        iter_serials_to_del = []
        for iter_serial, cached_iter in self.cached_iters.items():
            if cached_iter.num_groups_requested == cached_iter.num_groups_finished:
                iter_serials_to_del.append(iter_serial)
        for iter_serial in iter_serials_to_del:
            del self.cached_iters[iter_serial]

        self.current_iter_serial += 1
        new_iter_serial = self.current_iter_serial
        if self.opts.debug_out_queue: self.opts.debug_out_queue.put(f"[0] New iter {new_iter_serial}")

        cached_iter = self.CachedIter(new_iter_serial, iter(self.opts.sampler_iter), self.opts.sampler_iter.num_batches)
        self.cached_iters[new_iter_serial] = cached_iter
        self.global_lock.release()

        self.load_batches(cached_iter, self.opts.cached_batch_count + 1)

        return cached_iter
    
    def load_batches(self, cached_iter: CachedIter, num_batches):
        queued_any = False

        cached_iter.lock.acquire()
        new_num_groups_requested = min(cached_iter.num_groups_requested + num_batches, cached_iter.num_groups_total)
        if new_num_groups_requested != cached_iter.num_groups_requested:
            to_request = new_num_groups_requested-cached_iter.num_groups_requested
            self.work_queue.put(WorkItemLoadBatches(cached_iter, to_request))
            cached_iter.num_groups_requested = new_num_groups_requested
            if self.opts.debug_out_queue: self.opts.debug_out_queue.put(f"[0] Requesting {to_request} batches for iter {iter_serial}")
            queued_any = True
        cached_iter.lock.release()

        if self.opts.num_threads == 0 and queued_any:
            collator_threadproc(0, self)

    def shutdown(self):
        if self.threads_spawned:
            for _ in range(self.opts.num_threads): self.work_queue.put(WorkItem(WORK_TYPE_SHUTDOWN))

class WorkGroup:
    def __init__(self, cached_iter: "Collator.CachedIter", batch_serial, num_total):
        self.cached_iter = cached_iter
        self.batch_serial = batch_serial
        self.num_total = num_total
        self.gi_work_array: list[WorkItemGetItems] = []

        self.num_done_lock = threading.Lock()
        self.num_done = 0

    def __repr__(self): return f"{self.__class__.__name__}(iter_serial={self.iter_serial}, batch_serial={self.batch_serial}, "\
                                    f"num_done={self.num_done}, num_total={self.num_total})"

class CollatedResult:
    def __init__(self, batch_serial, result):
        self.batch_serial = batch_serial
        self.result = result

    def __repr__(self): return f"{self.__class__.__name__}({str(vars(self))})"

def collator_threadproc(thread_id: int, ctx: Collator):
    work_queue = ctx.work_queue
    global_lock = ctx.global_lock
    sub_batch_size = ctx.opts.sub_batch_size
    cached_iters = ctx.cached_iters
    getitem_func = ctx.opts.getitem_func
    collate_func = ctx.opts.collate_func
    debug_out_queue = ctx.opts.debug_out_queue

    if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Started")

    while 1:
        work: WorkItem = work_queue.get()
        if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Got {work}")
        work_type = work.type
        
        if work_type == WORK_TYPE_LOAD_BATCHES:
            lb_work: WorkItemLoadBatches = work
            
            cached_iter = lb_work.cached_iter
            
            cached_iter.lock.acquire()
            read_num_groups = cached_iter.num_groups_started
            
            cached_iter.num_groups_started += lb_work.num_batches
            array_of_batch_indices = list(itertools.islice(cached_iter.iter, lb_work.num_batches))
            cached_iter.lock.release()

            batches_spawned = 0
            for batch_indices in array_of_batch_indices:
                sub_batches = mins.chunkify(batch_indices, sub_batch_size)
                assert len(sub_batches)

                batch_serial = read_num_groups + batches_spawned
                work_group = WorkGroup(lb_work.cached_iter, batch_serial + 1, len(sub_batches))
                for sub_batch in sub_batches:
                    work_queue.put(WorkItemGetItems(work_group, sub_batch))
                    if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Queued {work_group.gi_work_array[-1]}")

                batches_spawned += 1


        elif work_type == WORK_TYPE_GET_ITEMS:
            gi_work: WorkItemGetItems = work
            work_group = gi_work.group

            gi_work.results = getitem_func(gi_work.indices)

            work_group.num_done_lock.acquire()
            work_group.num_done += 1
            work_group.num_done_lock.release()
            assert work_group.num_done <= work_group.num_total
            
            if work_group.num_done == work_group.num_total:
                assert work_group.num_done == len(work_group.gi_work_array)
                if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Completed group {work_group}")

                cached_iter = work_group.cached_iter
                
                cached_iter.lock.acquire()
                cached_iter.num_groups_finished += 1
                is_last_group = cached_iter.num_groups_finished == cached_iter.num_groups_total
                cached_iter.lock.release()

                collated = collate_func([gi_work.results for gi_work in work_group.gi_work_array])
                cached_iter.collated_queue.put(CollatedResult(work_group.batch_serial, collated))

                if is_last_group:
                    if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Completed iter {cached_iter.serial}")
                    cached_iter.collated_queue.put(CollatedResult(0, None))

                if thread_id == 0:
                    break
                

        elif work_type == WORK_TYPE_SHUTDOWN:
            break

        else: assert False

    if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Exiting")


class DataLoaderIter:
    def __init__(self, collator, cached_iter: Collator.CachedIter, collated_queue):
        self.collator = collator
        self.cached_iter = cached_iter
        self.collated_queue = collated_queue
        
        self.buffer: list[CollatedResult] = []
        self.next_batch_index = 1
    
    def __iter__(self):
        while 1:
            collated: CollatedResult = self.collated_queue.get()
            if not collated.batch_serial:
                assert not self.buffer # batch_serial == 0 should strictly come last
                break

            self.buffer.append(collated)

            found_i = next((i for i, x in enumerate(self.buffer) if x.batch_serial == self.next_batch_index), -1)
            if found_i != -1:
                to_yield = self.buffer[found_i].result
                self.buffer[found_i] = self.buffer[-1]
                self.buffer.pop()
                self.next_batch_index += 1
                
                yield to_yield
                self.collator.load_batches(self.cached_iter, 1)

class DataLoader:
    def __init__(self, ds, copts: COPTS):
        self.ds = ds
        self.collator = Collator(copts)

    def __del__(self):
        self.collator.shutdown()
    
    def __iter__(self):
        cached_iter = self.collator.start_new_iter()
        return iter(DataLoaderIter(self.collator, cached_iter, cached_iter.collated_queue))
    
    @classmethod
    def simple(cls, simple_ds: minds.SimpleDataset, sampler_iter_opts: mins.SIO = None, collator_opts: COPTS = None):
        sampler_iter_opts = sampler_iter_opts or mins.SIO()
        collator_opts = collator_opts or COPTS()

        collator_opts.sampler_iter = mins.Sampler(len(simple_ds)).iter(sampler_iter_opts)
        collator_opts.getitem_func = simple_ds.__getitem__
        collator_opts.collate_func = simple_collate_func
        return cls(simple_ds, collator_opts)

    @classmethod
    def hf(cls, hf_ds: hfds.Dataset, sampler_iter_opts: mins.SIO = None, collator_opts: COPTS = None):
        assert type(hf_ds) is hfds.Dataset, f"Dataset expected, not {type(hf_ds).__name__}"
        sampler_iter_opts = sampler_iter_opts or mins.SIO()
        collator_opts = collator_opts or COPTS()

        collator_opts.sampler_iter = mins.Sampler(len(hf_ds)).iter(sampler_iter_opts)
        collator_opts.getitem_func = hf_ds.__getitem__
        collator_opts.collate_func = HFCollate(hf_ds).__call__
        return cls(hf_ds, collator_opts)

In [4]:
import time
import random

def getitem_func(indices):
    #time.sleep(random.random())
    return [[i, -i] for i in indices]

dl = DataLoader(None, COPTS(mins.Sampler(11).iter(mins.SIO(5)), getitem_func, simple_collate_func, num_threads=4, cached_batch_count=1))

it1 = iter(dl)
it2 = iter(dl)
print(1, next(it1))
print(2, next(it2))
print(1, next(it1))
print(1, next(it1))
print(2, next(it2))
print(2, next(it2))
print()

for xb, yb in dl:
    print(xb, yb)
print("agane")
for xb, yb in dl:
    print(xb, yb)

1 ([0, 1, 2, 3, 4], [0, -1, -2, -3, -4])
2 ([0, 1, 2, 3, 4], [0, -1, -2, -3, -4])
1 ([5, 6, 7, 8, 9], [-5, -6, -7, -8, -9])
1 ([10, 3, 3, 6, 8], [-10, -3, -3, -6, -8])
2 ([5, 6, 7, 8, 9], [-5, -6, -7, -8, -9])
2 ([10, 10, 6, 5, 7], [-10, -10, -6, -5, -7])

[0, 1, 2, 3, 4] [0, -1, -2, -3, -4]
[5, 6, 7, 8, 9] [-5, -6, -7, -8, -9]
[10, 3, 8, 1, 4] [-10, -3, -8, -1, -4]
agane
[0, 1, 2, 3, 4] [0, -1, -2, -3, -4]
[5, 6, 7, 8, 9] [-5, -6, -7, -8, -9]
[10, 6, 9, 7, 2] [-10, -6, -9, -7, -2]


In [5]:
ds = minds.SimpleDataset(list(range(100)), list(range(0, -100, -1)))
dl = DataLoader.simple(ds, mins.SIO(16))
ds, vars(dl.collator), vars(dl.collator.opts)

(SimpleDataset(len=100, xs=int, ys=int),
 {'opts': <__main__.COPTS at 0x7f31b4c5e1d0>,
  'threads_spawned': False,
  'work_queue': <_queue.SimpleQueue at 0x7f31b4996520>,
  'global_lock': <unlocked _thread.lock object at 0x7f31b4c5e300>,
  'current_iter_serial': 0,
  'cached_iters': {}},
 {'sampler_iter': <minai.sampler.SamplerIter at 0x7f31b4c5e7d0>,
  'getitem_func': <bound method SimpleDataset.__getitem__ of SimpleDataset(len=100, xs=int, ys=int)>,
  'collate_func': <function __main__.simple_collate_func(array_of_results)>,
  'num_threads': 14,
  'cached_batch_count': 1,
  'sub_batch_divisor': 1.0,
  'debug_out_queue': None,
  'sub_batch_size': 16})

In [6]:
it = iter(dl)
print(next(it))
print(next(it))

it = iter(dl)
print(next(it))
print(next(it))

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15])
([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [-16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31])
([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15])
([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [-16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31])


In [7]:
dsd = minds.hf_load(minds.HF_DATASETS.FASHION_MNIST)
dst = dsd["train"]

Found cached dataset fashion_mnist (/home/nblzv/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
c = HFCollate(dst)
print(c)
print(c([dst[[0, 1]], dst[[2]]]))

HFCollate(features=('image', 'label'))
[[<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7F3190B4D5D0>, <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7F3190B4D6D0>, <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7F3190124FD0>], [9, 0, 0]]


In [9]:
for _ in range(2):
    for xs, ys in dl:
        print(xs, ys)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] [0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15]
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] [-16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]
[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47] [-32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63] [-48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63]
[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79] [-64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95] [-80, -81, -82, -83, -84, -85, -86, -87, -88, -89, -90, -91, -92, -93, -94, -95]
[96, 97, 98, 99, 79, 5, 86, 3, 86, 35, 37, 11, 84, 65, 9, 55] [-96, -97, -98, -99, -79, -5, -86, -3, -86, -35, -37, -11, -84, -65, -9, -55]
[0, 1

In [10]:
dl = DataLoader.hf(dst, mins.SIO(9, False))
next(iter(dl))

[[<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 [9, 0, 0, 3, 0, 2, 7, 2, 5]]

In [11]:
#e
class HFTransform:
    def __init__(self, features, transform, **extra_args):
        assert type(features) is hfds.features.features.Features
        self.features = tuple(features)
        self.transform = transform

        for k, v in extra_args.items():
            super().__setattr__(k ,v)

    def __call__(self, results):
        return self.transform(self, results)
    
    def __repr__(self):
        return f"HFTransform(features={list(self.features)})"

    @classmethod
    def ff_img_to_tensor(cls, features): # first_feature
        def tf(ctx: HFTransform, results):
            xs = results[ctx.features[0]]
            for i in range(len(xs)):
                xs[i] = TF.to_tensor(xs[i])
            return results
        
        return cls(features, tf)
    
    @classmethod
    def ff_img_decode_to_tensor(cls, features, half=False): # first_feature
        def tf(ctx: HFTransform, results):
            xs = results[ctx.features[0]]
            for i in range(len(xs)):
                raw = torch.frombuffer(xs[i]["bytes"], dtype=torch.uint8)
                decoded = TFIO.decode_image(raw)
                if ctx.half: decoded = decode.half()
                else: decoded = decoded.float()
                xs[i] = decoded / 255.0
            return results
        
        return cls(features, tf, half=half)

In [12]:
#e
def first(iterable):
    return next(iter(iterable))

def first_value(iterable):
    return next(iter(iterable.values()))

In [13]:
def to_tensor_transform(ctx, results):
    xs = results[ctx.features[0]]
    for i in range(len(xs)):
        xs[i] = TF.to_tensor(xs[i])
    return results

thds = dst.with_transform(HFTransform(dst.features, to_tensor_transform))
dl = DataLoader.hf(thds, mins.SIO(5, False))
first(dl)[0][0].shape

torch.Size([1, 28, 28])

In [14]:
tdst = dst.with_transform(HFTransform.ff_img_to_tensor(dst.features))
def bench(cached_batch_count, next_count):
    dl = DataLoader.hf(tdst, mins.SIO(64), collator_opts=COPTS(cached_batch_count=cached_batch_count))
    it = iter(dl)
    for _ in range(next_count): next(it)
print("prefetch 0")
%timeit -r 10 -n 1 bench(0, 1);
%timeit -r 10 -n 1 bench(0, 2);
%timeit -r 10 -n 1 bench(0, 3);
print("prefetch 1")
%timeit -r 10 -n 1 bench(1, 1); # For some reason this shows as slower when it is in fact pretty much the same time for the first batch to complete
%timeit -r 10 -n 1 bench(1, 2);
%timeit -r 10 -n 1 bench(1, 3);

prefetch 0
13.1 ms ± 614 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
20.3 ms ± 711 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
27.7 ms ± 890 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
prefetch 1
35.3 ms ± 1.75 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
59 ms ± 14.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
56.2 ms ± 10.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [20]:
tdst = dst.with_transform(HFTransform.ff_img_decode_to_tensor(dst.features)).cast_column("image", hfds.Image(decode=False))
def bench(cached_batch_count, next_count):
    dl = DataLoader.hf(tdst, mins.SIO(64), collator_opts=COPTS(cached_batch_count=cached_batch_count))
    it = iter(dl)
    for _ in range(next_count): 
        next(it)
print("prefetch 0")
%timeit -r 10 -n 1 bench(0, 1);
%timeit -r 10 -n 1 bench(0, 2);
%timeit -r 10 -n 1 bench(0, 3);
print("prefetch 1")
%timeit -r 10 -n 1 bench(1, 1); # For some reason this shows as slower when it is in fact pretty much the same time for the first batch to complete
%timeit -r 10 -n 1 bench(1, 2);
%timeit -r 10 -n 1 bench(1, 3);

prefetch 0
8.55 ms ± 551 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
12.5 ms ± 480 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
16.6 ms ± 476 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
prefetch 1
16.7 ms ± 837 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
28.4 ms ± 6.68 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
26.8 ms ± 2.43 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [16]:
#e
class DataLoaderDict(dict):
    def __init__(self, dataloaders_dict: dict[str, DataLoader]):
        super().__init__(dataloaders_dict)

    def __getitem__(self, key) -> DataLoader:
        return super().__getitem__(key)
        
    def __repr__(self):
        return f"DataLoaders({super().__repr__()})"

    @classmethod
    def hf(cls, dsd: hfds.DatasetDict, sampler_iter_opts: mins.SIO = None, collator_opts: COPTS = None):
        dls = {k: DataLoader.hf(dsd[k], sampler_iter_opts, collator_opts) for k in dsd}
        return cls(dls)

In [17]:
dls = DataLoaderDict.hf(dsd)
dls

DataLoaders({'train': <__main__.DataLoader object at 0x7f3164326ad0>, 'test': <__main__.DataLoader object at 0x7f31674ee2d0>})

In [19]:
import z_export
z_export.export()

Processing minai_nbs/datasets.ipynb -> minai/minai/datasets.py  |  same contents, skipping, took 0.001s
Processing minai_nbs/sampler.ipynb -> minai/minai/sampler.py  |  same contents, skipping, took 0.000s
Processing minai_nbs/setup+template.py -> minai/setup.py  |  same contents, skipping, took 0.000s
Processing minai_nbs/__init__+template.py -> minai/minai/__init__.py  |  same contents, skipping, took 0.000s
Processing minai_nbs/plot.ipynb -> minai/minai/plot.py  |  same contents, skipping, took 0.000s
Processing minai_nbs/mintils.py -> minai/minai/mintils.py  |  same contents, skipping, took 0.000s
Processing minai_nbs/data.ipynb -> minai/minai/data.py  |  same contents, skipping, took 0.000s

All done... took 0.003s
  lib_name: minai
  author: nblzv
  version: 0.1.1
